In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
original = cv2.imread('/content/cat.jpg', 0)

In [ ]:
plt.imshow(original, cmap='gray')

In [ ]:
templat = cv2.imread('/content/meomeo.png', 0)

In [ ]:
templat

In [ ]:
template= cv2.resize(templat, (55, 55))

In [ ]:
template

In [ ]:
# Edge detection
def edge_detection(image):
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(image, (5,5), 0)
    # Apply Canny edge detection
    edges = cv2.Canny(blur, 0, 50)
    return edges

In [ ]:
template_edge= edge_detection(template)

In [ ]:
edge_detection(template)

In [ ]:
target_edges = edge_detection(original)

In [ ]:
target_edges

In [ ]:
threshold = 0.2
scale_factors = np.linspace(0.8, 1.4, 100).tolist()  # Scale factors for template resizing

detections = []

for scale in scale_factors:
    # Resize the template for the current scale
    scaled_template = cv2.resize(template_edge, None, fx=scale, fy=scale)

    # Template matching
    res = cv2.matchTemplate(target_edges, scaled_template, cv2.TM_CCOEFF_NORMED)

    # Identify locations above the threshold
    loc = np.where(res >= threshold)

    for pt in zip(*loc[::-1]):
        detections.append((pt[0], pt[1], scaled_template.shape[1], scaled_template.shape[0]))

# Apply non-maximum suppression to avoid counting overlapping matches
def non_max_suppression(detections, overlapThresh):
    if len(detections) == 0:
        return []

    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in detections])

    # Initialize list of picked boxes
    pick = []

    x1 = rects[:, 0]
    y1 = rects[:, 1]
    x2 = rects[:, 2]
    y2 = rects[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Find overlap ratios
        xx1 = np.maximum(x1[i], x1[idxs[:-1]])
        yy1 = np.maximum(y1[i], y1[idxs[:-1]])
        xx2 = np.minimum(x2[i], x2[idxs[:-1]])
        yy2 = np.minimum(y2[i], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:-1]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return rects[pick].astype(int)

final_detections = non_max_suppression(detections, overlapThresh=0.1)

for (x1, y1, x2, y2) in final_detections:
    cv2.rectangle(target_edges, (x1, y1), (x2, y2), (255, 0, 0), 2)

print(f"Number of matches: {len(final_detections)}")

cv2.imwrite('detected_matches.jpg', target_edges)

In [ ]:
cv2.rectangle(target_edges, (x1, y1), (x2, y2), (255, 0, 0), 2)

In [ ]:
for (x1, y1, x2, y2) in final_detections:
    cv2.rectangle(original, (x1, y1), (x2, y2), (0, 0, 0), 2)

In [ ]:
cv2.rectangle(original, (x1, y1), (x2, y2), (0, 0, 0), 2)